Implementation of the UR robots inverse kinematics based on:

[Alternative Inverse Kinematic Solution of the UR5 Robotic Arm](https://link.springer.com/chapter/10.1007/978-3-030-90033-5_22)

In [ ]:
import numpy as np
np.set_printoptions(suppress=True) # Prettier printing

In [ ]:
eef_rotation_matrix = np.identity(3)
eef_rotation_matrix

In [ ]:
R = eef_rotation_matrix
r11, r12, r13 = R[0, :]
r21, r22, r23 = R[1, :]
r31, r32, r33 = R[2, :]

r11, r12, r13, r21, r22, r23, r31, r32, r33

In [ ]:
eef_translation = np.zeros(3)
t = eef_translation
px, py, pz = t
px, py, pz

In [ ]:
def calculate_theta1s2(r13, r23, px, py, d4, d6):
    arg1 = py - d6 * r23
    arg2 = px - d6 * r13
    denominator = np.sqrt(arg1**2 + arg2**2)
    term1 = np.arctan2(arg1, arg2) + np.pi / 2
    term2 = np.arccos(d4 / denominator)
    theta1_solution1 = term1 + term2 
    theta1_solution2 = term1 - term2
    return theta1_solution1, theta1_solution2

In [ ]:
def calculate_theta1s(r13, r23, px, py, d4, d6) -> tuple[float, float]:
    """Calculate the two solutions for theta1.
    Based on Equation (24) in the paper.
    """
    print("Args:", "r13, px, py, d4, d6")
    print("Args:", r13, px, py, d4, d6)
    A1 = px - d6 * r13
    B1 = d6 * r23 - py
    squared_term = A1 ** 2 + B1 ** 2 - d4 ** 2

    if not squared_term >= 0:
        raise ValueError("Not sure if/when this occurs for the UR robots, but the paper mentions this.")

    term1 = np.arctan2(A1, B1)
    # print(np.rad2deg(term1))
    term2 = np.arctan2(np.sqrt(squared_term), d4)
    # print(np.rad2deg(term2))
    theta1_solution1 = term1 + term2 
    theta1_solution2 = term1 - term2
    return theta1_solution1, theta1_solution2

In [ ]:
# C2 = a2
# 3 − a2
# 2 − d2
# 5 − (pz − d1) (2d5c234 + 2d6s5s234 + pz − d1)
# + (c1px + s1py) (2d5s234 − 2d6s5c234 − c1px − s1py) − d2
# 6s2
# 5


def calculate_theta2(px ,py, pz, d1, d5, d6, a2, a3, c1, s1, s5, c234, s234):
    A2 = 2 * a2 * (d1 - pz - (d5 * c234) - (d6 * s5 * s234))
    B2 = 2 * a2 * ((d5 * s234) - (d6 * s5 * c234) - (c1 * px) - (s1 * py))
    C2 = a3 ** 2 - a2 ** 2 - d5 ** 2 - (pz - d1) * (2 * d5 * c234 + 2 * d6 * s5 * s234 + pz - d1) + (c1 * px + s1 * py) * (2 * d5 * s234 - 2 * d6 * s5 * c234 - c1 * px - s1 * py) - d6 ** 2 * s5 ** 2
    theta2 = np.arctan2(A2, B2) + np.arctan2(np.sqrt(A2 ** 2 + B2 ** 2 - C2 ** 2), C2)
    print("Theta2", np.rad2deg(theta2))
    theta2_alt = np.arctan2(A2, B2) - np.arctan2(np.sqrt(A2 ** 2 + B2 ** 2 - C2 ** 2), C2)
    print("Theta2 alt", np.rad2deg(theta2_alt))
    return theta2

In [ ]:
def inverse_kinematics(eef_rotation_matrix: np.ndarray, eef_translation: np.ndarray, 
                        d1: float, d4: float, d5:float, d6: float, a2: float, a3: float):
    # Extracting the elements
    R = eef_rotation_matrix
    r11, r12, r13 = R[0, :]
    r21, r22, r23 = R[1, :]
    r31, r32, r33 = R[2, :]

    px, py, pz = eef_translation

    # Calculating the joints angles
    theta1, theta1_solution2 = calculate_theta1s(r13, r23, px, py, d4, d6)
    # print("Method 1 solutions for theta0", np.rad2deg(theta1), np.rad2deg(theta1_solution2))

    s1 = np.sin(theta1)
    c1 = np.cos(theta1)
    c5 = s1 * r13 - c1 * r23
    s5 = np.sqrt((s1*r11 - c1*r21)**2 + (s1*r12 - c1*r22)**2)
    theta5 = np.arctan2(s5, c5) # TODO negative is also solution

    s5 = np.sin(theta5)
    sign5 = np.sign(s5)

    c5 = np.cos(theta5)
    theta6 = np.arctan2(sign5 * (c1 * r22 - s1 * r12), sign5 * (s1 * r11 - c1 * r21))

    c6 = np.cos(theta6)
    s6 = np.sin(theta6)

    A234 = (c1 * r11) + (s1 * r21)
    theta234 = np.arctan2((c5 * c6 * r31) - (s6 * A234), (c5 * c6 * A234) + (s6 * r31))

    print(np.rad2deg(theta234))

    c234 = np.cos(theta234)
    s234 = np.sin(theta234)

    calculate_theta2(px ,py, pz, d1, d5, d6, a2, a3, c1, s1, s5, c234, s234)

    # KC = c1px + s1py − s234d5 + c234s5d6
    # KS = pz − d1 + c234d5 + s234s5d6

    KC = (c1 * px) + (s1 * py) - (s234 * d5) + (c234 * s5 * d6)
    KS = pz - d1 + (c234 * d5) + (s234 * s5 * d6)

    KSKC_term = (KS**2 + KC**2 - a2**2 - a3**2) / (2 * a2 * a3)
    theta3 = np.arctan2(np.sqrt(1 - KSKC_term**2), KSKC_term)

    print(np.rad2deg(theta1), np.rad2deg(theta3), np.rad2deg(theta5), np.rad2deg(theta6))

    # Kc = c2 * (a2 + c3 * a3) - s2 * (s3 * a3)

    # theta0_solution1, theta0_solution2 = calculate_theta1s2(r13, r23, px, py, d4, d6)
    # print("Method 2 solutions for theta0", np.rad2deg(theta0_solution1), np.rad2deg(theta0_solution2))

Testing with known poses.
In this support article by UR you can find the [DH parameters of the UR robots](https://www.universal-robots.com/articles/ur/application-installation/dh-parameters-for-calculations-of-kinematics-and-dynamics/).

Additionally, you can find spreadsheet there that calculates the forward kinematics of a UR5e.

All joints angles 45 degrees for the UR5e robots:

In [ ]:
eef_rotation45 = np.array([[-0.25000, -0.45711, 0.85355], [-0.95711, 0.25000, -0.14645], [-0.14645, -0.85355, -0.50000]])
eef_rotation45

In [ ]:
eef_translation45 = np.array([0.01662, -0.27149, -0.50952])
eef_translation45

In [ ]:
d1 = 0.1625
d4 = 0.1333
d5 = 0.0997
d6 = 0.0996
a2 = -0.425	# TODO check, in the paper these a's are the negated values of those on the UR site
a3 = -0.3922
inverse_kinematics(eef_rotation45, eef_translation45, d1, d4, d5, d6, a2, a3)
print("45 should be one of the solutions for all thetas")

All joint angles 0 for the UR5e:

In [ ]:
eef_rotation0 = np.array([[1, 0, 0], [0, 0, -1], [0, 1, 0]])
eef_rotation0

In [ ]:
eef_translation0 = np.array([-0.8172, -0.2329, 0.0628])
eef_translation0

In [ ]:
d1 = 0.1625
d4 = 0.1333
d5 = 0.0997
d6 = 0.0996
a2 = -0.425	
a3 = -0.3922
inverse_kinematics(eef_rotation0, eef_translation0, d1, d4, d5, d6, a2, a3)
print("0 should be one of the solutions for all thetas")

Forward kinematics

In [ ]:
def dh_matrix(theta_i, d_i, a_i, alpha_i):
    """Calculate the DH matrix for a given joint."""
    return np.array([
        [np.cos(theta_i), -np.sin(theta_i) * np.cos(alpha_i), np.sin(theta_i) * np.sin(alpha_i), a_i * np.cos(theta_i)],
        [np.sin(theta_i), np.cos(theta_i) * np.cos(alpha_i), -np.cos(theta_i) * np.sin(alpha_i), a_i * np.sin(theta_i)],
        [0, np.sin(alpha_i), np.cos(alpha_i), d_i],
        [0, 0, 0, 1]
    ])


d_values = [d1, 0, 0, d4, d5, d6]
a_values = [0, a2, a3, 0, 0, 0]
alpha_values = [np.pi / 2, 0, 0, np.pi / 2, -np.pi / 2, 0]
theta_i = np.deg2rad(45) # same for all joints


pose = np.identity(4)
for i in range(6):
    d_i = d_values[i]
    a_i = a_values[i]
    alpha_i = alpha_values[i]
    pose = pose @ dh_matrix(theta_i, d_i, a_i, alpha_i)

print(pose)